# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
Classification. Students must be grouped into two classes: _needs intervention_ and _does not need intervention_. The output variable does not take on continuous values.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [127]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("https://raw.githubusercontent.com/udacity/machine-learning/master/projects/student_intervention/student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [128]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data["passed"] == "yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data["passed"] == "no"])

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 0.67%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [129]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [130]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
print X_all.head()

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
   school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  famsize_GT3  \
0          1          0      1      0   18          0          1            1   
1          1          0      1      0   17          0          1            1   
2          1          0      1      0   15          0          1            0   


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [131]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state = 42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
I found it quite difficult to decide on appropriate models for this problem, based on what I've learned so far. As far as I can tell there are two pieces of information I can base my decision on:
- Size of the data set: 395. This is pretty small so I think I have to look for algorithms that can handle this.
- Number of features: 48. This is pretty large compared to the size of the data set.

If there is additional information I am overlooking, I would love to get feedback on that. The course material sofar has not provided me with the tools to pick the 'right' algorithm based on a provided dataset (or maybe I have overlooked that part of course).

The most important characteristic of this data set is its size. A small data set requires a model with low complexity or high bias. A complex model will capture irregularities in training data and does not generalize well to testing data. So I think the focus should be on algorithms that (can) produce simple models.

I have chosen the following three algorithms for this problem:
- Logistic Regression
- Support Vector Machines
- Random Forest

What I would actually do in a real-world situation is to try out a larger number of different models and pick the best one for the job. Besides, I think 48 features for 395 data points is rather much (curse of dimensionality) so I would try and eliminate some features using [feature selection](http://scikit-learn.org/stable/modules/feature_selection.html) in sklearn.

###### Logistic Regression
Logistic Regression is one of the simplest models available so I think it should always be the first model to try out. If it gives good (enough) results there is either no need to look further or you have a baseline to compare other, more complex algorithms to.

A disadvantage of logistic regression is that there should be no high intercorrelations (multicollinearity) among the predictors. I have not checked whether this is the case for the current data set. Another disadvantage is that the model that is produced may be too simple (since it can only be a linear surface).

A real-world examples:
- [Using Binary Logistic Regression to Investigate High Employee Turnover](http://blog.minitab.com/blog/real-world-quality-improvement/using-binary-logistic-regression-to-investigate-high-employee-turnover)

###### Support Vector Machines
SVM's are still effective in high-dimensional spaces. 48 features isn't really high-dimensional but compared to the size of the training set it is. SVM's are memory-efficient, although for a training set of this size that's not really an issue. Most importantly I think is that SVM's are flexible. Even though there is a risk of overfitting, this can be mitigated by tuning the `C` and `gamma` parameters. Choosing a different kernel may also improve results. I'm not sure yet what kernel would be best for this problem based on the data.

So even though an SVM may not perform best with default parameters on the given data set, improvements may be possible in the tuning phase. It may be interesting to see what's possible.

Real-world examples:
- [Support vector machine classification and validation of cancer tissue samples using microarray expression data](https://www.ncbi.nlm.nih.gov/pubmed/11120680)
- [Applications of Support Vector
Machines in Chemistry](http://www.ivanciuc.org/Files/Reprint/Ivanciuc_Applications_of_Support_Vector_Machines_in_Chemistry.pdf)

###### Random Forest
The random forest algorithm is based on randomized decision trees. Decision trees are prone to overfitting (low bias, high variance) the deeper they become. A random forest counters this problem by averaging over a large number of trees.

A disadvantage of the algorithm is runtime performance. Both training and predicting takes time due to the large number of models (trees) that is used.

Real-world examples:
- [Application of Random Forests Methods to Diabetic Retinopathy Classification Analyses](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4062420/)

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [132]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [133]:
# Import the three supervised learning models from sklearn
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble

# Initialize the three models
clf_A = linear_model.LogisticRegression(random_state = 42)
clf_B = svm.SVC(random_state = 42)
clf_C = ensemble.RandomForestClassifier(random_state = 42)

# Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train
y_train_300 = y_train

# Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0023 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8593.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7647.
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0026 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8562.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7914.
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0053 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8468.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.8060.
Training a SVC using a training set size of 100. . .
Trained model in 0.0019 seconds
Made predictions in 0.0013 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7746.
Training a SVC using a training set size of 200. . .
Trained mo

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Logistic Regression **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0022                  | 0.0004                 | 0.8593           | 0.7647          |
| 200               | 0.0038                  | 0.0003                 | 0.8562           | 0.7914          |
| 300               | 0.0062                  | 0.0006                 | 0.8468           | 0.8060          |

** Support Vector Machine **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0024                  | 0.0021                 | 0.8777           | 0.7746          |
| 200               | 0.0054                  | 0.0025                 | 0.8679           | 0.7815          |
| 300               | 0.0098                  | 0.0028                 | 0.8761           | 0.7838          |

** Random Forest **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0341                  | 0.0014                 | 0.9844           | 0.7153          |
| 200               | 0.0242                  | 0.0010                 | 0.9964           | 0.7857          |
| 300               | 0.0243                  | 0.0011                 | 0.9951           | 0.8092          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
Most notable in the results is the F1 score of the random forest algorithm on the training set compared to the test set. This indicates that the random forest overfits on the training set. I do not have a good explanation for this. The default for `max_features` is `sqrt(48)` so there should be quite some randomization over the feature space. I tried with a larger number of trees (100 and 1000) but that only increases the F1 score on the training set to `1.0000` without improving on the test set. This is the main reason for not choosing the random forest. There is probably a good reason the random forest overfits but I'm not sure what it is.

Another reason for not choosing a random forest is training costs. Its about five times slower that the other two algorithms.

Also noteworthy is the F1 score improvement on the test set when more training data is provided. This is the case for all three algorithms.

The other two models have comparable performance and costs although prediction time for SVM is slower. However, I think that SVM offers more possibilities for optimization. Therefore, I choose the SVM for further investigation.

As I said before I have trouble explaining how the dataset relates to algorithm choice.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
We choose a Support Vector Machine (SVM) for further investigation. A SVM is a classification algorithm that tries to find the optimal boundary between two classes, based on the provided features. It does this by locating so-called support vectors. A support vector is a sample from the feature space that is close to the boundary between the positive (_passed_) and the negative (_failed_) samples. In this case a sample from the feature space is a student.

So a SVM tries to find a boundary between students that have passed and students that have failed by attempting to find students that are closest to this boundary. When these students, the _support vectors_, are found, the boundary can be determined by drawing a line between passing and failing students that is as far away from both the passed as the failed side as possible.

When the model must make a prediction for an unknown student, the algorithm simply checks on what side of the boundary this student is and reports _passing_ or _failing_ accordingly.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [134]:
# Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Create the parameters list you wish to tune
n_features = len(X_all.columns)
c_params = [0.1, 0.5, 1.0, 5.0, 10.0]
parameters = [
    { 'kernel': ['poly', 'rbf', 'sigmoid'],
      'C': c_params,
      'gamma': [ 1.0 / (n_features * 32), 1.0 / (n_features * 16), 1.0 / (n_features * 8),
                 1.0 / (n_features * 4), 1.0 / (n_features * 2), 1.0 / n_features ]},
    { 'kernel': ['linear'],
      'C': c_params }
]

# Initialize the classifier
clf = clf_B

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label = 'yes')

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring = f1_scorer, n_jobs = -1)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print grid_obj.best_params_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

{'kernel': 'rbf', 'C': 10.0, 'gamma': 0.0013020833333333333}
Made predictions in 0.0049 seconds.
Tuned model has a training F1 score of 0.8529.
Made predictions in 0.0017 seconds.
Tuned model has a testing F1 score of 0.7891.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The result is quite disappointing. The SVM performs only marginally better with tuned parameters. The scores are:
- 0.8529 on the training set and
- 0.7891 on the test set

My conclusion is that either:
1. SVM is not a good match for this particular problem or
2. I picked the wrong set of parameters for GridSearchCV or
3. data scaling is an issue as described [here](http://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use)

I'm going to investigate number 3 first.

**Possible improvement: scaling **

Column values have different scales in this data set. For example, `age` ranges from `15` to `22` while `school_GP` is `0` or `1`. So I will scale each column to be between 0 and 1 and try again.

In [135]:
# Scale all columns in training and test set.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = scaler.fit_transform(X_test[X_test.columns])

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = GridSearchCV(clf, parameters, scoring = f1_scorer, n_jobs = -1)
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print grid_obj.best_params_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


{'kernel': 'sigmoid', 'C': 10.0, 'gamma': 0.020833333333333332}
Made predictions in 0.0053 seconds.
Tuned model has a training F1 score of 0.8452.
Made predictions in 0.0029 seconds.
Tuned model has a testing F1 score of 0.7862.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.